<a href="https://colab.research.google.com/github/tphamdo/tphamdo.github.io/blob/master/US_Pollution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Table of Content :

1) [Data curation, parsing, and management](#data_curation)

2) [Exploratory data analysis](#eda)

3) [Hypothesis testing and ML to provide analysis](#hyp_testing)

4) [Insights](#insights)

<a class="anchor" id="data_curation"></a>
## 1. Data curation, parsing, and management

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [4]:
df = pd.read_csv("./input/pollution_us_2000_2016.csv")

### Peek through the dataset

-- Remove unnecessary columns --

Index column (Unnamed: 0) serves no purpose.

State Code, Country Code, Site Num, Address are removed since State, Country and City columns are sufficent.

All the units columns (NO2 Units, etc.) are removed since they only have 1 unique value representing their unit.

In [5]:
df = df.drop(['Unnamed: 0','State Code','County Code','Site Num','Address','NO2 Units','O3 Units','SO2 Units','CO Units'],axis=1)

In [6]:
df.head()

,State,County,City,Date Local,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,Arizona,Maricopa,Phoenix,2000-01-01,19.041667,49.0,19,46,0.022500,0.040,10,34,3.000000,9.0,21,13.0,1.145833,4.2,21,NaN
1,Arizona,Maricopa,Phoenix,2000-01-01,19.041667,49.0,19,46,0.022500,0.040,10,34,3.000000,9.0,21,13.0,0.878947,2.2,23,25.0
2,Arizona,Maricopa,Phoenix,2000-01-01,19.041667,49.0,19,46,0.022500,0.040,10,34,2.975000,6.6,23,NaN,1.145833,4.2,21,NaN
3,Arizona,Maricopa,Phoenix,2000-01-01,19.041667,49.0,19,46,0.022500,0.040,10,34,2.975000,6.6,23,NaN,0.878947,2.2,23,25.0
4,Arizona,Maricopa,Phoenix,2000-01-02,22.958333,36.0,19,34,0.013375,0.032,10,27,1.958333,3.0,22,4.0,0.850000,1.6,23,NaN


<a class="anchor" id="eda"></a>
## 2. Exploratory data analysis

<a class="anchor" id="hyp_testing"></a>
## 3. Hypothesis testing and ML to provide analysis

<a class="anchor" id="insights"></a>
## 4. Insights